## Run the training session for the yolo algorithm

In [45]:
# !python ../yolov5_model/yolov5/train.py --img 640 --batch 16 --epochs 8 --data dataset.yaml --weights ../yolov5_model/yolov5/yolov5s.pt

## Run the validation session for the yolo algorithm

In [46]:
# !python ../yolov5_model/yolov5/detect.py --weights ../yolov5_model/yolov5/runs/train/exp6/weights/best.pt --img 640 --conf 0.4 --source ../data/validation/ --save-txt --project yolo_worms_validation

In [47]:
# Retrieve the results of the detection stage
import os
import pandas as pd

detection_directory = './yolo_worms_validation/exp/labels/'

all_dfs = []
for f in os.listdir(detection_directory):
    label_df = pd.read_csv(f'{detection_directory}{f}', sep=' ', names=["target", 'xcenter', 'ycenter', 'width', 'height'])
    label_df = label_df.value_counts('target').rename_axis('worm_type').reset_index(name='target')
    label_df.insert(0, 'image_id', f[:-4])
    all_dfs.append(label_df)

resulting_df = pd.concat(all_dfs).reset_index(drop=True)
resulting_df['worm_type'] = resulting_df['worm_type'].apply(lambda x: 'pbw' if x == 1 else 'abw')
resulting_df

,image_id,worm_type,target
0,id_c1cf5ae2d07679bae328ea3d,pbw,2
1,id_e1195008d2e0774e771e106a,abw,7
2,id_bdec925b6ec3208e023c15cc,pbw,12
3,id_1ee4af25d3434cfa63084a41,pbw,13
4,id_10a46e521341e1c5fd4d3754,pbw,5
...,...,...,...
699,id_22398ef7b7431f22462a88aa,pbw,9
700,id_2e59a2a183e19d979ca9c452,pbw,86
701,id_5d3e8dadf1eab86097a980fd,abw,6
702,id_11c9a092c1c360925a4e89bc,abw,1


In [48]:
# Read in the training df
import pandas as pd

train_df = pd.read_csv('../data/train.csv')
train_df['image_id_worm'] = train_df['image_id_worm'].apply(lambda x: os.path.splitext(x)[0])
train_df

,image_id_worm,worm_type,number_of_worms
0,id_0002ea6f15c7fa6f4c221783,pbw,51
1,id_0005ef295aafe6acc63587db,pbw,8
2,id_00084298dd030a500033ff78,NaN,0
3,id_00093f2c76f6488737325859,pbw,12
4,id_000b2e6c437c643f25d4a6c3,pbw,87
...,...,...,...
9895,id_ffe6e193dd78b7258d864f6b,abw,5
9896,id_ffe98feb1e592e887b81c3f0,pbw,95
9897,id_ffef90ecf47baa38434f84fb,pbw,26
9898,id_fff06bb2e59bfce437ac9fc0,NaN,0


In [49]:
# Read in the validation dataset

validation_df = pd.read_csv('../data/validation_set.csv')
validation_df = validation_df.drop(columns=['worm_type','number_of_worms','image_width','image_height','worm_code'])
validation_df['image_id_worm'] = validation_df['image_id_worm'].apply(lambda x: os.path.splitext(x)[0])

cleaned_validation_df = pd.merge(validation_df, train_df, left_on="image_id_worm", right_on="image_id_worm")
cleaned_validation_df

,image_id_worm,worm_type,number_of_worms
0,id_8006a1fb857496f0db4bf8f2,pbw,5
1,id_640fc3e82cda70ec750d5e7f,NaN,0
2,id_11fb2d19e5a2597e593b35f2,pbw,1
3,id_f73ab7b153f2028ae790798c,pbw,13
4,id_f73ab7b153f2028ae790798c,abw,2
...,...,...,...
989,id_b025312965b1475de026b89b,pbw,11
990,id_777c1f76f0f868d52ccf6570,abw,8
991,id_fc4d24bdf0a33e1821a1d75d,pbw,11
992,id_935a60ecb40448df0812f438,NaN,0


In [50]:
# Full join
full_df = pd.merge(cleaned_validation_df, resulting_df, left_on=["image_id_worm", "worm_type"], right_on=["image_id", "worm_type"], how="outer")
full_df['target'] = full_df['target'].fillna(0)
full_df['number_of_worms'] = full_df['number_of_worms'].fillna(0)
full_df.to_csv('../results/yolo_validation_results.csv', index=False)
full_df['difference'] = (full_df['target'] - full_df['number_of_worms']).abs()
full_df

,image_id_worm,worm_type,number_of_worms,image_id,target,difference
0,id_8006a1fb857496f0db4bf8f2,pbw,5.0,id_8006a1fb857496f0db4bf8f2,5.0,0.0
1,id_640fc3e82cda70ec750d5e7f,NaN,0.0,NaN,0.0,0.0
2,id_11fb2d19e5a2597e593b35f2,pbw,1.0,id_11fb2d19e5a2597e593b35f2,1.0,0.0
3,id_f73ab7b153f2028ae790798c,pbw,13.0,id_f73ab7b153f2028ae790798c,10.0,3.0
4,id_f73ab7b153f2028ae790798c,abw,2.0,NaN,0.0,2.0
...,...,...,...,...,...,...
1070,NaN,pbw,0.0,id_e69f39987fb77e1a73c528d2,1.0,1.0
1071,NaN,abw,0.0,id_46f39916d246b7ccec87a410,1.0,1.0
1072,NaN,pbw,0.0,id_07db35f11b042e68e8019cc7,1.0,1.0
1073,NaN,pbw,0.0,id_e8cd26f34b22d40963cb7359,1.0,1.0


In [51]:
# Evaluate quality of model on validation data
full_df['difference'].sum() / len(validation_df)

3.8449691991786445

In [52]:
# Convert to actual format
resulting_df

,image_id,worm_type,target
0,id_c1cf5ae2d07679bae328ea3d,pbw,2
1,id_e1195008d2e0774e771e106a,abw,7
2,id_bdec925b6ec3208e023c15cc,pbw,12
3,id_1ee4af25d3434cfa63084a41,pbw,13
4,id_10a46e521341e1c5fd4d3754,pbw,5
...,...,...,...
699,id_22398ef7b7431f22462a88aa,pbw,9
700,id_2e59a2a183e19d979ca9c452,pbw,86
701,id_5d3e8dadf1eab86097a980fd,abw,6
702,id_11c9a092c1c360925a4e89bc,abw,1


In [54]:
final_validation_df = resulting_df.copy(deep=True)

final_validation_df['Image_ID'] = (final_validation_df['image_id'] + "_" + final_validation_df['worm_type']).str.upper()
final_validation_df['Target'] = final_validation_df['target']
final_validation_df = final_validation_df.drop(columns=['image_id', 'worm_type', 'target'])
final_validation_df.to_csv('../results/yolo_results.txt', index=False, sep='\t')
final_validation_df

,Image_ID,Target
0,ID_C1CF5AE2D07679BAE328EA3D_PBW,2
1,ID_E1195008D2E0774E771E106A_ABW,7
2,ID_BDEC925B6EC3208E023C15CC_PBW,12
3,ID_1EE4AF25D3434CFA63084A41_PBW,13
4,ID_10A46E521341E1C5FD4D3754_PBW,5
...,...,...
699,ID_22398EF7B7431F22462A88AA_PBW,9
700,ID_2E59A2A183E19D979CA9C452_PBW,86
701,ID_5D3E8DADF1EAB86097A980FD_ABW,6
702,ID_11C9A092C1C360925A4E89BC_ABW,1


## Run the testing session for the yolo algorithm

In [55]:
!python ../yolov5_model/yolov5/detect.py --weights ../yolov5_model/yolov5/runs/train/exp6/weights/best.pt --img 640 --conf 0.4 --source ../data/testing/ --save-txt --project yolo_worms_testing

detect: weights=['../yolov5_model/yolov5/runs/train/exp6/weights/best.pt'], source=../data/testing/, data=../yolov5_model/yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolo_worms_testing, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-210-gf9bb984 Python-3.9.13 torch-1.12.1 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/2803 /Users/sampastoriza/Documents/Programming/DataScienceDevelopment/590FinalProject/data/testing/id_00332970f80fa9a47a39516d.jpg: 480x640 4 pbws, 60.9ms
image 2/2803 /Users/sampastoriza/Documents/Programming/DataScienceDevelopment/590FinalProject/data/testing/id_0035981bc3ae42eb5b57a317.jpg: 640x480 20 abws, 59.3ms


In [56]:
# Retrieve the results of the detection stage
import os
import pandas as pd

detection_directory = './yolo_worms_testing/exp/labels/'

all_dfs = []
for f in os.listdir(detection_directory):
    label_df = pd.read_csv(f'{detection_directory}{f}', sep=' ', names=["target", 'xcenter', 'ycenter', 'width', 'height'])
    label_df = label_df.value_counts('target').rename_axis('worm_type').reset_index(name='target')
    label_df.insert(0, 'image_id', f[:-4])
    all_dfs.append(label_df)

resulting_test_df = pd.concat(all_dfs).reset_index(drop=True)
resulting_test_df['worm_type'] = resulting_test_df['worm_type'].apply(lambda x: 'pbw' if x == 1 else 'abw')
resulting_test_df

,image_id,worm_type,target
0,id_fa1c3e9aeebd2987c28d0c29,pbw,129
1,id_d79c7291e33f998d91147ffe,abw,12
2,id_4886fcd185bb443e06069cff,pbw,1
3,id_e8fb1659b5e21747118a99de,pbw,1
4,id_31e85cca8d8fbb709572961e,pbw,1
...,...,...,...
2061,id_f72c2e70efed021099f91f77,abw,3
2062,id_f72c2e70efed021099f91f77,pbw,2
2063,id_fc3b7bd391cc95a072ba1dea,pbw,8
2064,id_043550ebb988a8aebb842551,pbw,2


In [102]:
test_df = pd.read_csv('../data/test.csv')

abw_df = test_df.copy(deep=True)
abw_df['worm_type'] = 'abw'
pbw_df = test_df.copy(deep=True)
pbw_df['worm_type'] = 'pbw'

full_test_df = pd.concat([abw_df, pbw_df], ignore_index=True)
full_test_df['image_id_worm'] = full_test_df['image_id_worm'].apply(lambda x: os.path.splitext(x)[0])
full_test_df['image_id_worm'] = (full_test_df['image_id_worm'] + "_" + full_test_df['worm_type']).str.lower()
full_test_df = full_test_df.drop(columns=['worm_type'])
full_test_df = full_test_df.rename(columns={"image_id_worm": "Image_ID"})
full_test_df

,Image_ID
0,id_00332970f80fa9a47a39516d_abw
1,id_0035981bc3ae42eb5b57a317_abw
2,id_005102f664b820f778291dee_abw
3,id_0066456f5fb2cd858c69ab39_abw
4,id_007159c1fa015ba6f394deeb_abw
...,...
5601,id_ffad8f3773a4222f8fe5ba1a_pbw
5602,id_ffb65e6de900c49d8f2ef95a_pbw
5603,id_ffbcb27fa549278f47505515_pbw
5604,id_ffc0e41e10b0c964d4a02811_pbw


In [109]:
final_testing_df = resulting_test_df.copy(deep=True)

final_testing_df['Image_ID'] = final_testing_df['image_id']
final_testing_df['Image_ID'] = final_testing_df['Image_ID'].apply(lambda x: os.path.splitext(x)[0])
final_testing_df['Image_ID'] = (final_testing_df['Image_ID'] + "_" + final_testing_df['worm_type']).str.lower()
final_testing_df = pd.merge(full_test_df, final_testing_df, on="Image_ID", how="left")

final_testing_df = final_testing_df.drop(columns=['image_id', 'worm_type'])
final_testing_df['target'] = final_testing_df['target'].fillna(0)
final_testing_df['target'] = pd.to_numeric(final_testing_df['target'], downcast='integer')
final_testing_df = final_testing_df.rename(columns={"Target": "target"})
final_testing_df = final_testing_df.sort_values(by='Image_ID').reset_index(drop=True)

final_testing_df.to_csv('../results/yolo_test_results.csv', index=False)
final_testing_df

,Image_ID,target
0,id_00332970f80fa9a47a39516d_abw,4
1,id_00332970f80fa9a47a39516d_pbw,0
2,id_0035981bc3ae42eb5b57a317_abw,0
3,id_0035981bc3ae42eb5b57a317_pbw,20
4,id_005102f664b820f778291dee_abw,9
...,...,...
5601,id_ffbcb27fa549278f47505515_pbw,0
5602,id_ffc0e41e10b0c964d4a02811_abw,0
5603,id_ffc0e41e10b0c964d4a02811_pbw,0
5604,id_fff8c253115aacded09ad7ed_abw,0


In [ ]:
final_testing_df

In [105]:
resulting_test_df

,image_id,worm_type,target
0,id_fa1c3e9aeebd2987c28d0c29,pbw,129
1,id_d79c7291e33f998d91147ffe,abw,12
2,id_4886fcd185bb443e06069cff,pbw,1
3,id_e8fb1659b5e21747118a99de,pbw,1
4,id_31e85cca8d8fbb709572961e,pbw,1
...,...,...,...
2061,id_f72c2e70efed021099f91f77,abw,3
2062,id_f72c2e70efed021099f91f77,pbw,2
2063,id_fc3b7bd391cc95a072ba1dea,pbw,8
2064,id_043550ebb988a8aebb842551,pbw,2
